
# 0 - Imports and defining functions

In [ ]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

import os

import glob

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

def export_results(p2p, name):
    # open file in write mode
    with open(r'data_out/'+ name + '.txt', 'w') as f_export_init:
        for item in p2p:
            # write each item on a new line
            f_export_init.write("%s\n" % item)
        print('Exported' + name + 'point to point mapping \n')



# 1 - Data loading

In [ ]:
# Input folder name
inputFolder = 'COLLECTION_001'
# Concatenate folder name with the path
inputPath = '../data/' + inputFolder + '/'
filesList = []

# Iterate directory
for file in os.listdir(inputPath):
    # check only text files
    if file.endswith('.off'):
        filesList.append(file)

loadLandmark = True # If set to True, load the landmark data from the input folder for each mesh

# 2 - Computing the functional maps

**Add correspondences**

In [ ]:
for sourceFile in filesList:
    sourceFileShort = os.path.splitext(sourceFile)[0]
    print(f'====='+ sourceFileShort + '=====\n')
    for targetFile in filesList:
        targetFileShort = os.path.splitext(targetFile)[0]
        if sourceFile == targetFile:
            continue

        print(f'--------------------------\n' +
        'Processing p2p mapping: from ' + sourceFile + ' to ' + targetFile + 
        '\n--------------------------\n')

        mesh1 = TriMesh(inputPath + targetFile, area_normalize=True, center=True)
        mesh2 = TriMesh(inputPath + sourceFile, area_normalize=True, center=True)

        if loadLandmark:
            landmarks_mesh1 = np.loadtxt(inputPath + targetFileShort + '_landmarks.txt',dtype=int)[:10]  # loading N landmarks
            landmarks_mesh2 = np.loadtxt(inputPath + sourceFileShort + '_landmarks.txt',dtype=int)[:10]  # loading N landmarks
            landmarks = np.column_stack([landmarks_mesh1, landmarks_mesh2])
            #landmarks[:,0] = landmarks_mesh1
            #landmarks[:,1] = landmarks_mesh2
            print(f'landmarks: ' + str(landmarks) + '\n')

        
        else:
                landmarks_mesh1 = []
                landmarks_mesh2 = []


        # ----- Compute descriptors
        process_params = {
            'n_ev': (25,25),  # Number of eigenvalues on source and Target
            'landmarks': landmarks,  # loading 5 landmarks
            'subsample_step': 5,  # In order not to use too many descriptors
            'descr_type': 'HKS',  # WKS or HKS
        }


        model = FunctionalMapping(mesh1,mesh2)
        model.preprocess(**process_params,verbose=True);

        # ----- Fit the model
        fit_params = {
            'w_descr': 1e0, #scaling for the descriptor preservation term
            'w_lap': 1e-2, #scaling of the laplacian commutativity term
            'w_dcomm': 1e-1, #scaling of the multiplicative operator commutativity
            'w_orient': 0, #scaling of the orientation preservation term
            #'orient_reversing':True, #Whether to use the orientation reversing term instead of the orientation preservation one
            #'optinit':'zeros' #Initialization
        }

        model.fit(**fit_params, verbose=True)

        # ----- p2p map visualization
        p2p_21 = model.get_p2p(n_jobs=1)
        #cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
        #double_plot(mesh1,mesh2,cmap1,cmap2)

        # ----- ICP refinement
        #model.icp_refine(verbose=True)
        #p2p_21_icp = model.get_p2p()
        #cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_icp]
        #double_plot(mesh1,mesh2,cmap1,cmap2)

        # ----- ZoomOut refinement
        model.change_FM_type('classic') # We refine the first computed map ('classic') or the icp-refined one ('icp')
        model.zoomout_refine(nit=5, step = 1, verbose=True)
        print(model.FM.shape)
        p2p_21_zo = model.get_p2p()
        cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
        double_plot(mesh1,mesh2,cmap1,cmap2)

        # ----- Export results
        # check if data_out folder exists, if not create it
        if not os.path.exists('data_out'):
            os.makedirs('data_out')

        # export point to point mappings if they exist

        # initial point to point mapping
        if 'p2p_21' in locals():
            export_results(p2p_21, 'map_'+ sourceFileShort + '_' + targetFileShort)

        # icp refined point to point mapping
        if 'p2p_21_icp' in locals():
            export_results(p2p_21_icp, 'map_'+ sourceFileShort + '_' + targetFileShort + '_icp')

        # zoomout refined point to point mapping
        if 'p2p_21_zo' in locals():
            export_results(p2p_21_zo, 'map_'+ sourceFileShort + '_' + targetFileShort + '_zo')
